In [1]:
!mkdir /tmp/submission

import os, sys

#os.system("pip install -U -t /kaggle/working/submission \"bitsandbytes==0.43.1\" accelerate \"peft==0.11.1\" \"numpy==1.26.4\"")
#os.system("pip install -U -t /kaggle/working/submission \"transformers==4.42.4\"")
os.system("pip download transformers==4.43.3 -d /tmp/submission")
os.system("pip download bitsandbytes==0.43.2 -d /tmp/submission")
os.system("pip download accelerate==0.32.1 -d /tmp/submission")
os.system("pip download peft==0.12.0 -d /tmp/submission")
os.system("pip download numpy==1.26.4 -d /tmp/submission")

#os.system("pip cache purge")
#os.system("cp -a /kaggle/working/submission/transformers /kaggle/working/submission/transformersnew")
sys.path.insert(0, "/tmp/submission")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 79.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.2/751.2 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.5/776.5 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.5/435.5 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7

In [2]:
!cp -r /kaggle/input/mistral-nemo-12b-4-bit/transformers/instruction-tuned/1/Mistral-Nemo-Instruct-12b-4bit /tmp/submission
!cp -r /kaggle/input/20q-mistral-nemo-lora-adapters/transformers/default/8/FINAL_weights_for_real /tmp/submission

In [3]:
%%writefile /tmp/submission/main.py

import os
import sys
import subprocess
import shutil
import glob
import importlib.util
import traceback
import pickle
import socket
import psutil
import time

# Host and port configuration for the socket connection
host = "127.0.0.1"

KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if not os.path.exists(KAGGLE_AGENT_PATH):
    KAGGLE_AGENT_PATH = "/tmp/submission/"
    os.environ['PYTHONPATH'] = f"{KAGGLE_AGENT_PATH}:{os.environ.get('PYTHONPATH', '')}"
sys.path.insert(0, KAGGLE_AGENT_PATH)

def install_transformers():
    install_command = [
        sys.executable, '-m', 'pip', 'install',
        '--no-index', '--upgrade', '--force-reinstall',
        '--no-deps', f'--find-links={KAGGLE_AGENT_PATH}',
        'transformers==4.43.3', 'bitsandbytes==0.43.2', 'accelerate==0.32.1', 'peft==0.12.0', 'huggingface_hub'
    ]
    subprocess.run(install_command, check=True)

install_transformers()

def find_free_port():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(('', 0))
        return s.getsockname()[1]

def is_port_in_use(port):
    """ Check if the given port is in use """
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        return s.connect_ex(('127.0.0.1', port)) == 0

def terminate_existing_servers(port):
    """ Terminate any processes that are using the given port """
    for proc in psutil.process_iter(['pid', 'name']):
        try:
            for conns in proc.connections(kind='inet'):
                if conns.laddr.port == port:
                    proc.kill()
        except psutil.AccessDenied:
            continue

# Find a free port
port = find_free_port()

# Ensure there are no existing servers using port
terminate_existing_servers(port)

# Start the background process
process = subprocess.Popen([sys.executable, KAGGLE_AGENT_PATH + "main2.py", str(port)])

def agent_fn(obs, cfg):
    data = pickle.dumps((obs, cfg))
    
    # Retry configuration
    retries = 10
    delay = 6  # seconds
    while retries:
        try:
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
                time.sleep(1)
                s.connect(('127.0.0.1', port))
                s.sendall(data)
                result_data = s.recv(4096)
                result = pickle.loads(result_data)
                return result
        except ConnectionRefusedError:
            print(f"Connection refused, retrying in {delay} seconds...")
            time.sleep(delay)
            retries -= 1
        except ConnectionResetError:
            print(f"Connection reset by peer, returning 'yes'")
            return "yes"
    
    raise ConnectionError("Failed to connect to the server after multiple attempts")

Writing /tmp/submission/main.py


In [4]:
%%writefile /tmp/submission/main2.py

import pickle
import socket


################
# Determine if submit or commit and import necessary packages

import os, sys
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
VERBOSE = False
os.environ['PYTHONPATH'] = "/tmp/submission:" + os.environ.get('PYTHONPATH', '')
if not os.path.exists(KAGGLE_AGENT_PATH):
    KAGGLE_AGENT_PATH = "/tmp/submission/"
    VERBOSE = True

import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    PreTrainedTokenizerBase
)
from peft import PeftModel

sys.path.insert(0, KAGGLE_AGENT_PATH)   


#################
# Configure and load model into memory

from dataclasses import dataclass
import re
import sklearn
import numpy as np
import pandas as pd

host = "127.0.0.1"  # Localhost
port = int(sys.argv[1])  # Port passed as an argument

@dataclass
class Config:
    model_name = f"{KAGGLE_AGENT_PATH}Mistral-Nemo-Instruct-12b-4bit"
    lora_dir =  f"{KAGGLE_AGENT_PATH}FINAL_weights_for_real"
    max_length = 1024
    batch_size = 1
    device = torch.device("cuda")    
    tta = False  
    spread_max_length = False  

cfg = Config()

#Load Mistral NeMo
tokenizer = AutoTokenizer.from_pretrained(cfg.model_name, add_bos_token = True)

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quanty_type = "fp4", 
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quanty = True,
)

model = AutoModelForCausalLM.from_pretrained(
    cfg.model_name,
    quantization_config = bnb_config,
    torch_dtype = torch.float16,
    device_map = "auto",
    attn_implementation="eager",
    trust_remote_code = True,
    use_cache = False
)


#Add LoRA Adapters
model = PeftModel.from_pretrained(model, cfg.lora_dir)


#################
# Prompts and tools for processing the game state

import random

list_prev_words = ['acacia', 'accent chair', 'acorn', 'adhd medication', 'adjustable bench', 'Advertisement', 'aerator', 'Agave', 'agave plant', 'Air compressor', 'Air Conditioner', 'Air filter', 'air handler', 'air plant', 'air tank', 'Air vent', 'airport lighting', 'Alarm system', 'alfalfa', 'almond milk', 'aluminum foil', 'amazon echo', 'analog stick', 'Analogy', 'anchovy', 'Anemone', 'Anesthesia', 'angelfish', 'ankle bracelet', 'antidepressant', 'antiseptic wipe', 'aperol spritz', 'apple', 'Apple pie', 'applesauce', 'apricot tree', 'Aprons', 'aquarium salt', 'archery target', 'arrival board', 'aspirin', 'Atmosphere', 'attic ladder', 'audio guide', 'autograph', 'automatic door', 'backgammon board', 'Backrest', 'bacon', 'Bacteria', 'baggage conveyor', 'Baguette', 'balance beam', 'bald eagle', 'ballpoint pen', 'bamboo', 'bamboo leaf', 'banana', 'bandana', 'bank statement', 'bar towel', 'barbell collar', 'Barber Chair', 'barcode', 'Bat', 'Bath Mat', 'battery', 'battery bank', 'beach chair', 'beach umbrella', 'bean', 'bean plant', 'Beanbag', 'bear', 'Bed frame', 'bedspread', 'beech tree', 'beer coaster', 'beer glass', 'beer stein', 'beer tap', 'Bike', 'binder clip', 'Binoculars', 'bird netting', 'Bird seed', 'Blinds', 'boa constrictor', 'Board Games', 'Bobby Pins', 'Bollards', 'Bonding Agent', 'bonsai tree', 'book cover', 'bookbag', 'Bookcase', 'bookend', 'Bookends', 'Bottled Water', 'bowl', 'bracelet', 'Bread knife', 'bread maker', 'Bread pudding', 'Brewery merchandise', 'Briefcase', 'Brochure', 'brochure stand', 'broken glass', 'broth', 'Brownies', 'brush', 'brussel sprout', 'buckle', 'Bug spray', 'Bulb', 'bulletin', 'bumble bee', 'bumper', 'Bumper sticker', 'bunk bed', 'Bunsen burner', 'butter', 'butter knife', 'butterfly', 'butterfly net', 'button', 'cable tie', 'cable tray', 'cactus flower', 'cake', 'calcite', 'calculator', 'camp stove', 'candle holder', 'candy wrapper', 'cane', 'canning jar', 'cannoli', 'Cantaloupe', 'car antenna', 'car engine', 'car seat', 'carbon', 'Card', 'cardboard', 'Cardboard box', 'cardboard tube', 'caribou', 'carnation', 'Cash Register', 'casket', 'Cat carrier', 'catalytic converter', 'catheter', 'Cauliflower', 'cedar', 'Ceiling fan', 'centerpiece', 'centipede', 'Cereal', 'cereal bar', 'chair cushion', 'Champagne flute', 'Chandelier', 'charger', 'charging cord', 'Cheesecake', 'cherry tree', 'Chessboard', 'Chew toy', 'chicken wire', 'chiller', 'chime', 'chimney', 'chimpanzee', 'chocolate bar', 'Chocolate cake', 'cholla cactus', 'christmas tree', 'chutney', 'cilantro', 'cinder block', 'Cinnamon roll', 'clay', 'climber', 'climbing rope', 'clip', 'clock', 'clog', 'clothes rack', 'clutch bag', 'coal crusher', 'Coat Rack', 'cobblestone', 'cobweb', 'cockroach', 'cocktail glass', 'coconut', 'coffee bag', 'Coffee beans', 'Coffee Grinder', 'coffee ground', 'Coffee grounds', 'coffee machine', 'Coffee Makers', 'coffee mug', 'coffee table', 'coffin', 'coke', 'colander', 'cold pack', 'Comic Book', 'common milkweed', 'compactor', 'compass', 'compost', 'concentrator', 'concrete block', 'concrete wall', 'condenser', 'conditioner', 'conference table', 'Contact lenses', 'Conveyor belt', 'cooler', 'Cooling Tower', 'cordless phone', 'corkscrew', 'cotton', 'cotton candy', 'cowboy boot', 'crayfish', 'Cream cheese', 'crochet hook', 'Croissant', 'cruise ship', 'crutch', 'cuff', 'culture medium', 'cupcake', 'curler', 'curling', 'curling iron', 'Curtains', 'cuticle', 'Cutlery', 'cutter', 'Cutting Board', 'cypress knee', 'daisy', 'damselfish', 'Dandelion', 'dashboard camera', 'dead leaf', 'decal', 'Deciduous tree', 'denim', 'denim jeans', 'dental chair', 'deposit slip', 'Desk Chairs', 'desk lamp', 'desktop computer', 'Diaper', 'diary', 'dill pickle', 'dining chair', 'dinosaur skeleton', 'Dish soap', 'dishcloth', 'dishtowel', 'dishwasher', 'Disinfectant', 'Diving board', 'dj mixer', 'dj turntable', 'docket', 'dog poop', 'Dog shampoo', 'dog tag', 'dogwood', 'dolphin', 'domino', 'Donuts', 'door', 'doughnut', 'Drain Hose', 'drainage pipe', 'drapery', 'Drapes', 'dresser', 'drill bit', 'drying rack', 'duckweed', 'ducky', 'duct', 'Duct tape', 'Duffle Bags', 'Dumbbells', 'Dump truck', 'dust jacket', 'Duvet', 'duvet cover', 'DVD', 'DVD Player', 'dye', 'Dynamite', 'ear drop', 'Ear protection', 'Earl Grey tea', 'earplug', 'earring', 'earthworm', 'Edamame', 'Edible flowers', 'elastic bandage', 'elbow pad', 'electric bike', 'electric fan', 'electric motorcycle', 'electric oven', 'electric razor', 'electric saw', 'electric scooter', 'electric screwdriver', 'electric toothbrush', 'electric violin', 'Electrical panel', 'electrical tape', 'electronic keyboard', 'electronic piano', 'Elevator', 'Elliptical Trainers', 'embroidery', 'Emergency Exit Sign', 'emergency light', 'emergency lighting', 'Emergency lights', 'empty can', 'enamel', 'enamel pin', 'end table', 'Energy drink', 'engagement ring', 'engine oil', 'Engravings', 'entrance door', 'envelope', 'eruption', 'Escalators', 'espresso machine', 'Eucalyptus', 'evaporator', 'excavator', 'exercise mat', 'Exhaust Fan', 'exhibit sign', 'Exit sign', 'Extension Cord', 'extension ladder', 'eye', 'eye mask', 'eyeglass', 'fabric roll', 'fabric sofa', 'Face mask', 'Facial Toner', 'fallen log', 'fallen tree', 'fan blade', 'Fanny pack', 'fanta', 'feather boa', 'fence gate', 'fence panel', 'fennel', 'fern', 'Fertilizer', 'file folder', 'finger', 'Finger food', 'fir', 'Fire alarm', 'fire coral', 'Fire Escape Ladder', 'Fire Extinguisher', 'fire helmet', 'fire hydrant', 'fireplace mantle', 'First Aid Kit', 'fish scale', 'fishing net', 'fishing rod', 'fishnet', 'Fitness tracker', 'flashlight', 'flat shoe', 'flea', 'Floor jacks', 'Floor Mats', 'flour', 'flower crown', 'foam roller', 'Fog machine', 'foil paper', 'Food bowl', 'Food warmers', 'footbridge', 'footrest', 'Fortune cookie', 'fossil', 'fox', 'frappe', 'Frappuccino', 'Free Weights', 'French toast', 'fresheners', 'Fridge magnet', 'Fried rice', 'frisbee', 'frozen meat', 'frozen waffle', 'fruit cup', 'fudge brownie', 'fuel filter', 'Fungi', 'fur', 'Furniture', 'Furniture polish', 'Fuse', 'Gadget', 'game board', 'Garbage bag', 'Garbage Disposal', 'garbage truck', 'garden fork', 'garden rake', 'garden shovel', 'garment rack', 'garment steamer', 'garter', 'gas cap', 'Gas Mask', 'gas stove', 'gas turbine', 'gasket', 'gator', 'gecko', 'gel', 'gelatin', 'Generator', 'ghost crab', 'gift bag', 'gift box', 'gin', 'ginger', 'ginger ale', 'giraffe', 'glass figurine', 'glass panel', 'glass sheet', 'glass shelf', 'Glass Table', 'glass window', 'glitter', 'Glove', 'gold', 'Golf Cart', 'goose', 'gown', 'GPS', 'graduated cylinder', 'granola', 'grape', 'grape juice', 'Grape Vine', 'Grapefruit', 'Graphic Novel', 'Graphing Calculator', 'Gravity', 'green alga', 'Green beans', 'Greeting Card', 'grill', 'grinder', 'groundhog', 'guitar pick', 'Guitar String', 'guppy', 'gutter', 'Gym Mats', 'Habanero pepper', 'hacksaw', 'Hair clip', 'hair clipper', 'Hair Dryer', 'hair elastic', 'Hair Gel', 'hair tie', 'Hairbrush', 'Hairspray', 'hairspray can', 'ham', 'hamburger helper', 'hamster', 'Hamster wheel', 'Hamstring Curl Machine', 'hand drill', 'Hand dryer', 'hand lotion', 'hand pruner', 'Hand Sanitizer', 'Handbag', 'handcuff', 'handlebar', 'Handrail', 'hanging basket', 'hanging hook', 'Hard hat', 'Hash browns', 'hat', 'hay bale', 'Hazelnut', 'hazelnut tree', 'HDMI cable', 'headband', 'headboard', 'Headlamp', 'headlight', 'headphone', 'Headphones', 'Hearing aid', 'Heart Rate Monitors', 'Heating Element', 'Hibiscus', 'High Chair', 'highlighter', 'Hiking boots', 'hole puncher', 'honey', 'Honeybee', 'honeycrisp', 'honeydew', 'honeydew melon', 'hose line', 'hospital bed', 'Hot dog', 'hot tea', 'Hot water bottle', 'house key', 'humidity meter', 'hummus', 'humus', 'hvac system', 'hydrant', 'hydro turbine', 'Ice cream maker', 'ice crusher', 'Ice cube tray', 'ice pop', 'ice sculpture', 'ice tea', 'ice tray', 'Ice Water', 'Iced coffee', 'Icicle', 'igloo', 'ignition system', 'impact crusher', 'impact wrench', 'impatiens', 'Incense', 'Index Card', 'infrared sensor', 'Inhaler', 'ink cartridge', 'Insulation', 'internet modem', 'internet router', 'Interstate', 'inverter', 'invoice', 'iris', 'Iron', 'Iron bar', 'Ironing Board', 'Irrigation system', 'IV', 'iv bag', 'jack stand', 'jacket', 'jaffa cake', 'jam', 'jane', 'jasmine', 'jello', 'Jewelry Box', 'Journal', 'Joystick', 'juice box', 'juicer', 'Jumper Cables', 'junction box', 'juniper bush', 'juniper tree', 'Kale smoothie', 'Karaoke machine', 'kelp', 'ketchup', 'Key chain', 'key ring', 'keyring', 'Khaki pants', 'kheer', 'kickstand', 'king snake', 'Kitchen cabinet', 'kitchen knife', 'kiwifruit', 'knapweed', 'knitting needle', 'Komodo dragon', 'Lab coat', 'laboratory manual', 'lace', 'lace dress', 'lace top', 'Lanyard', 'Laptop', 'Laser beam', 'Lathe', 'Latte', 'laundry basket', 'laundry cart', 'lawn mower', 'Lawnmower', 'laxative', 'leaf', 'leaf rake', 'leash', 'legal pad', 'Lego', 'lemming', 'lemon balm', 'lemon juice', 'lemon wedge', 'lemon zest', 'leopard', 'leotard', 'level sensor', 'Library Card', 'library stamp', 'license plate', 'lid', 'lifter', 'lifting belt', 'lifting crane', 'Light Switch', 'Lightbulb', 'lighting rig', 'lily', 'lime', 'limousine', 'linen', 'lint brush', 'Lint Roller', 'lint trap', 'lipstick', 'Litter scooper', 'livestock trailer', 'loose change', 'Lottery Ticket', 'lug', 'Luggage', 'magnet', 'mail truck', 'maintenance truck', 'mango', 'mango tree', 'mangrove seedling', 'manicure set', 'maple tree', 'Marshmallow', 'martini', 'Masks', 'Matches', 'maxi dress', 'Measuring cup', 'measuring spoon', 'Measuring Tape', 'meatloaf', 'medallion', 'medical cart', 'medical chart', 'medical gown', 'Medical Records', 'Medication', 'medication dispenser', 'melon', 'Menorah', 'merchandise rack', 'mesquite tree', 'Metal detector', 'metal file', 'metal foil', 'metal lid', 'metal spoon', 'mic stand', 'microphone stand', 'Microscope', 'Microwave', 'migraine', 'milkweed', 'milling', 'Milling machine', 'Mini fridge', 'miniskirt', 'Mixing Bowl', 'mixing console', 'Mobile phone', 'model airplane', 'Modem', 'moleskin', 'mop', 'mortar', 'moss ball', 'motion sensor', 'motor', 'motorcycle', 'Motorcycle Helmet', 'mount saint lias', 'Mouse', 'mousepad', 'Movie poster', 'moving walkway', 'mri', 'mri machine', 'mucus', 'Muffin', 'Muffin Tin', 'Muffler', 'Mural', 'muscle relaxant', 'Mushroom', 'Nachos', 'Nail clipper', 'Nail polish', 'Nail Polish Remover', 'Name tag', 'Nap mat', 'Necklace', 'needle', 'neon light', 'netgear', 'nightshade', 'nile monitor', 'nintendo switch', 'nitrogen', 'nitrous', 'noise-canceling headphones', 'nokia phone', 'noodle', 'northern cardinal', 'northern mockingbird', 'nose', 'nose ring', 'Notebook', 'Nozzle', 'nylon', 'Oak table', 'oak tree', 'oakley sunglasses', 'oar', 'oatmeal cookie', 'Ocarina', 'office chair', 'oil filter', 'Ointment', 'Olive Oil', 'Onyx', 'operating table', 'orange juice', 'Orange peel', 'orange slice', 'Orange tree', 'Orange zest', 'ostrich', 'ottoman', 'Outdoor Furniture', 'Oven door', 'Oven rack', 'overhead wire', 'owl', 'oxygen', 'Oxygen tank', 'oyster', 'Pacifier', 'Packing peanuts', 'Pad', 'pail', 'paint roller', 'paint sprayer', 'paint thinner', 'paintbrush', 'Pajamas', 'Pallet Jack', 'palmetto tree', 'Pamphlet', 'pant', 'Pants', 'paper roll', 'Paper towels', 'Paperweights', 'Paperwork', 'parachute', 'Parallel bars', 'park map', 'parking pass', 'parrot', 'parrotfish', 'parsley', 'parts cleaner', 'pea plant', 'pea soup', 'peace lily', 'peach tree', 'peanut butter', 'pear tree', 'pedal boat', 'peeler', 'pelican', 'pellet', 'Pencil Sharpener', 'penguin', 'peony', 'pepper spray', 'peppermint', 'Perfume', 'perfume bottle', 'Personal protective equipment', 'Pesticide', 'petroleum', 'petunia', 'phone case', 'Phone charger', 'Photo album', 'photocopier', 'piano', 'piano stool', 'Pickaxe', 'picket', 'picnic basket', 'picture frame', 'pilates', 'pilates reformer', 'pillow sham', 'pillowcase', 'pillowtop', 'pine cone', 'pineapple', 'pipe', 'pipette', 'Piping tips', 'piranha', 'pitcher', 'pizza box', 'Placemat', 'plant saucer', 'planter', 'Planter Box', 'Plaque', 'plastic baggie', 'plastic chair', 'plastic fork', 'Plastic Gloves', 'plastic pallet', 'plastic pipe', 'plastic sign', 'plastic tub', 'plastic tube', 'Plastic wrap', 'Plate', 'Play mat', 'Playing cards', 'Pliers', 'plunger', 'polar bear', 'Polyester', 'pomegranate', 'Pool cleaner', 'Popcorn', 'Popcorn machine', 'porcupine', 'Portable speaker', 'possum', 'postage stamp', 'pot holder', 'Potato', 'potato chip', 'potted plant', 'power adapter', 'power cable', 'power cord', 'Power Drill', 'power grid', 'Power lines', 'power meter', 'power pole', 'Power strip', 'prairie sunflower', 'Pressure gauge', 'projector', 'Protein bar', 'protein shake', 'Protein Shakes', 'Pry Bar', 'Public Address System', 'pudding', 'pumpkin', 'Pumpkin pie', 'punching bag', 'purse', 'push pin', 'Puzzle', 'Quesadilla', 'Rabbit', 'raccoon', 'Radio scanner', 'Radish', 'Rags', 'rail track', 'rainboot', 'rainbow cake', 'Raindrops', 'raisin', 'ramen', 'rangefinder', 'raspberry', 'Rat', 'reading glasses', 'reading pillow', 'receipt printer', 'Red velvet cake', 'Red wine', 'redwood tree', 'refrigerator', 'Reindeer', 'relief valve', 'Relish', 'Reptile', 'Resin', 'Respirator', 'rhubarb', 'ribbon', 'rice krispie', 'Rice pudding', 'ricotta', 'Rivet', 'roadrunner', 'Robot arm', 'rocketship', 'rocking chair', 'roller', 'rollercoaster', 'Rolling pin', 'rooster', 'Roots', 'rose', 'rose bush', 'rosemary', 'Router', 'Rowing machines', 'rubber', 'ruby', 'rucksack', 'Rug', 'ruler', 'rusty nail', 'RV', 'safety glasses', 'safety goggles', 'Safety harness', 'safety railing', 'Salad Dressing', 'salt scrub', 'salt shaker', 'saltwater', 'sandal', 'sandhill crane', 'sandstorm', 'sash', 'Satellite', 'satellite dish', 'saucer', 'sauerkraut', 'school bus', 'Scones', 'Scoreboard', 'Scrap metal', 'Scratching posts', 'screen', 'Screwdriver', 'scrubber', 'sea turtle', 'seafoam', 'seagrass', 'seagull', 'seahorse', 'sealant', 'sealing machine', 'seam', 'security gate', 'Security systems', 'sedge', 'Seed packet', 'seedling tray', 'seltzer', 'serving bowl', 'serving platter', 'sewage system', 'Sewing Kit', 'sewing needle', 'shaker', 'sheetrock', 'Shin Guards', 'shingle', 'shipping container', 'Shipping label', 'shirt', 'shoelace', 'shop towel', 'shovel', 'Shower caddy', 'shower cap', 'shower chair', 'shower mat', 'Sieve', 'silicon', 'Silicone', 'Silicone mat', 'Silver ore', 'Sippy cup', 'Sketchbook', 'skewer', 'Sleeping bag', 'sling', 'slit', 'smartboard', 'Smartphone', 'Smartwatch', 'smith machine', 'smoke alarm', 'Smoke Detector', 'smoothie', 'Snacks', 'snake', 'snowball', 'snowdrift', 'snowmobile', 'Soap Dispenser', 'Soap Suds', 'soccer ball', 'sock', 'Soda', 'sofa', 'solar panel', 'Soldering iron', 'sorting machine', 'Sound system', 'soundboard', 'soup', 'Soybeans', 'Space probe', 'spare bulb', 'Spatula', 'spider web', 'spirit level', 'Sportswear', 'Spotlight', 'spray', 'Sprinkler', 'Sprinkler System', 'squash', 'squid', 'Squirrel', 'Stage lights', 'Stain Remover', 'Stained glass window', 'Stair Stepper', 'staircase', 'stationary bike', 'steak', 'steak knife', 'steam iron', 'steel sheet', 'steering wheel', 'stick', 'Sticker', 'Sticky notes', 'Stove', 'straightening iron', 'strainer', 'Street sign', 'streetlamp', 'strobe light', 'stroller', 'stump', 'Sugar packet', 'Suitcase', 'sumantri', 'sunglass', 'sunglasses', 'Sunscreen', 'support beam', 'surgical drape', 'survey marker', 'sushi', 'suture', 'swamp milkweed', 'swedish fish', 'sweet tea', 'sweetgum tree', 'swimsuit', 'Swing set', 'sword fern', 'sycamore tree', 'syrup', 'Tablet', 'tackle box', 'Tank', 'tank top', 'tape dispenser', 'tape measure', 'Taser', 'tassel', 'tea bag', 'tea cup', 'tea leaf', 'teddy bear', 'telephone', 'Telescope', 'Television', 'television remote', 'Tennis ball', 'test strip', 'thermal underwear', 'thermometer probe', 'throw blanket', 'throw rug', 'thumbtack', 'thyroid', 'thyroid medication', 'Ticket Machine', 'Ticket stub', 'tile', 'tin foil', 'tiramisu', 'tire', 'tire gauge', 'tire inflator', 'tire iron', 'Tissue box', 'toad', 'Toaster Oven', 'Tofu', 'toilet brush', 'Toiletries bag', 'Tomato', 'tomato plant', 'tongue', 'Tongue Depressor', 'Tool Belt', 'Tool Box', 'Toothbrush', 'Toothpaste', 'toothpick', 'tote bin', 'Touchscreen', 'tow strap', 'Tow truck', 'Towel', 'towel basket', 'Tracksuit', 'Tractor', 'traffic signal', 'Train brake', 'Train door', 'train horn', 'training cone', 'transmission tower', 'Trash bag', 'travel pillow', 'trellis', 'trench coat', 'Tricycle', 'trimming scissors', 'trinket box', 'trophy', 'trouser', 'Truck', 'trumpet', 'Tulip', 'tuning fork', 'tupperware', 'Turn signal', 'Turnstiles', 'turtle', 'Tweezers', 'twine', 'Udon Noodles', 'ukulele case', 'ultrasound', 'Ultraviolet Lamp', 'Umbrella stand', 'unbuttoned jacket', 'underwater camera', 'underwater volcano', 'Unleavened Bread', 'unripe banana', 'unripe fruit', 'upholstered sofa', 'upright piano', 'Urinal', 'USB drives', 'usb stick', 'user manual', 'utility belt', 'Utility Box', 'Utility Cart', 'vacuum cleaner', 'valve', 'Vanilla Extract', 'Vanity mirror', 'Vans', 'varnish', 'Vase', 'vegan chocolate', 'Vegetable', 'vegetarian chili', 'Vegetation', 'Veggie Burger', 'velvet couch', 'velvet ribbon', 'Velvet rope', 'velvet rug', 'Vending Machine', 'ventilation fan', 'Ventilation System', 'vespa', 'vest', 'Video Camera', 'vietnamese coffee', 'vintage clock', 'vintage dollhouse', 'violin', 'violin bow', 'Violin Strings', 'Virus', 'voice recorder', 'volcanic ash', 'volkswagen', 'volleyball', 'vortex mixer', 'Voucher', 'VR Headset', 'walking stick', 'Wall Art', 'Wall Decorations', 'wall mirror', 'wall shelf', 'washcloth', 'Washing machine', 'wasp', 'wastewater', 'Water', 'Water Bottle', 'water bowl', 'Water Buffalo', 'water chestnut', 'water dish', 'Water Fountain', 'Water Heater', 'water hose', 'water hyacinth', 'water jug', 'water pump', 'water snake', 'water tank', 'water well', 'water wing', 'Watering Can', 'waterslide', 'wax', 'weasel', 'webcam', 'wedding ring', 'weighing scale', 'weight plate', 'weightlifting belt', 'Wheelchair ramp', 'whisk', 'Whistle', 'whiteboard', 'wig', 'wii', 'wild garlic', 'wind chime', 'wind vane', 'Windbreak', 'window cleaner', 'windsock', 'Wine Aerator', 'wine decanter', 'wine glass', 'Wine Opener', 'Wine Tasting Kit', 'wing', 'wiper', 'wire', 'wire brush', 'wire hanger', 'Wire rack', 'wireless microphone', 'Wireless Speaker', 'wisteria', 'wonton', 'wonton soup', 'wood chip', 'wooden spoon', 'woodpecker', 'woolen blanket', 'workout log', 'Wrench', 'Wrist Weights', 'Wristband', 'writing paper', 'yarrow', 'yeast', 'yoga block', 'Yoga mat', 'Yogurt', 'yogurt container', 'yule log', 'zinc', 'zip tie', 'ziploc']
OLD_KEYWORDS = [word.lower() for word in list_prev_words]

class MistralPromptFormatter:
    def __init__(self, inference_mode = True):
        self.sys_prompt = "You are an AI assistant playing the 20 Questions game. In this game the Answerer is given a secret keyword. "\
        "The Questioner then asks yes-or-no questions regarding the keyword, and the Answerer answers them accurately. "\
        "Then the Guesser tries to guess the keyword based on the questions and answers in the game.\n"
        self.inference_mode = inference_mode
        
    def format_instruction(self, instruction):
        return f"[INST]{instruction}[/INST]"
    
    def format_obs(self, obs, guesses):
        if obs.turnType == 'ask':
            return self.ask_format(obs, guesses)
        if obs.turnType == 'answer':
            return self.answer_format(obs, guesses)
        if obs.turnType == 'guess':
            return self.guess_format(obs, guesses)

    def similar_guess_format(self, guess):
        sim_prompt = ("You are assisting the game by generating similar keywords to a given keyword. The similar keyword you generate"
                      "should be either a more specific instance of the keyword, or a thing of a very similar kind as the keyword. Here are some examples:\n"
                      "Keyword: Apple. Similar Keyword: Fuji apple\n"
                      "Keyword: Ibuprofen. Similar Keyword: Tylenol\n"
                      "Keyword: Brownie. Similar Keyword: Fudge brownie\n"
                      "Keyword: Polar bear. Similar Keyword: Black bear\n"
                      "Keyword: Steel. Similar Keyword: Aluminum\n"
                      "Keyword: Curtain. Similar Keyword: Persian blinds\n"
                      "Keyword: HDMI cable. Similar Keyword: USB cable\n"
                      "Keyword: CPU. Similar Keyword: GPU\n"
                      "Keyword: Duck. Similar Keyword: Goose\n"
                      "Keyword: Yoga. Similar keyword: Pilates\n"
                      "Keyword: Car. Similar keyword: Sedan\n"
                      "Keyword: Dumbbell. Similar keyword: Barbell\n"
                      f"Now it is your turn. Output ONLY the similar keyword and nothing else, without punctuations.\n Keyword: {guess}. Similar Keyword:")
        formatted_conversation = self.format_instruction(self.sys_prompt+sim_prompt)
        return formatted_conversation
        
    def ask_format(self, obs, guesses):
        ask_prompt = ("You are playing this game as the Questioner. Ask yes-or-no questions that narrow down what the keyword could be. The keyword is a specific thing, NOT a place and NOT a person. "
                      "DO NOT ask if the keyword is a specific thing, rather ask something about the keyword, as in the following examples:\n"
                      "Example 1: Do NOT ask: 'Is the keyword knife?', INSTEAD ask: 'Is it a knife used for a particular purpose?'\n"
                      "Example 2: Do NOT ask: 'Is the keyword cow?', INSTEAD ask: 'Is it a specific type or breed of cow?'\n"
                      "Example 3: Do NOT ask: 'Is it bottle?', INSTEAD ask: 'Is it a bottle made of a specific material?'\n"
                      "Example 4: Do NOT ask: 'Is the keyword lamp?', INSTEAD ask: 'Is it a type of lamp?'\n"
                      "Do NOT assume the game has ended, the game will determine when to stop. Instead keep asking questions about the keyword. "
                      "Do not output any text other than the question. Start with broad questions "
                      "and based on the answers so far narrow down your questions. Now ask your first question.\n\nQuestion: ")
        formatted_conversation = self.format_instruction(self.sys_prompt + ask_prompt) 
        for i in range(len(obs.questions)):
            formatted_conversation += obs.questions[i] + "</s>"
            if self.inference_mode or i != len(obs.questions)-1: 
                formatted_conversation += '\n' + self.format_instruction(f"Answer: {obs.answers[i]}\nAsk your next question. Remember to NOT ask if the keyword is a specific thing, but INSTEAD ask something about the keyword.\nQuestion:")  
        return formatted_conversation.strip()
    
    def guess_format(self, obs, guesses):
        guess_prompt = "You are playing this game as the Guesser. After each question and answer you will guess what the keyword is based on the knowledge you have gained "\
        "from the questions and answers about the keyword. Do NOT give repeated guesses.\n\nRound 1:\nQuestion: " + obs.questions[0]+'\nAnswer: '+ obs.answers[0] +".\nNow guess the keyword."
        formatted_conversation = self.format_instruction(self.sys_prompt + guess_prompt) 
        for i in range(len(guesses)):
            formatted_conversation += guesses[i] + "</s>"
            if self.inference_mode or i < len(guesses)-1: 
                formatted_conversation += '\n' + self.format_instruction(f"Round {i+2}: \nQuestion: "+obs.questions[i+1]+"\nAnswer: "+obs.answers[i+1]+".\n"
                                                                         "Now guess the keyword based on the information from all rounds about the keyword. Do not give repeated guesses.\nGuess:")  
        return formatted_conversation.strip()
    
    def answer_format(self, obs, guesses):
        answer_prompt = "You are playing this game as the Answerer. You will answer accurately the questions regarding the keyword with ONLY yes or no. "\
        "If the questioner did not ask a question, simply answer 'yes'. "\
        f"For this game the keyword is {obs.keyword}. Now answer the question about the keyword.\nKeyword: {obs.keyword}.\nQuestion: " + obs.questions[0] +"\nAnswer:"
        formatted_conversation = self.format_instruction(self.sys_prompt + answer_prompt) 
        for i in range(len(obs.answers)):
            formatted_conversation += obs.answers[i] + "</s>"
            if i != len(obs.questions)-1: 
                formatted_conversation += '\n' + self.format_instruction(f"Answer the following question about the keyword.\nKeyword: {obs.keyword}. \nQuestion: " + obs.questions[i+1]+"\nAnswer:")  
        return formatted_conversation.strip()
    
    
    def format_response(self, string, string_len, obs):
        response = string[string_len+3:]
        response = response.replace('</s>', '')
        if obs.turnType in ['guess', 'ask']:
            return response
        else:
            response = response.lower()
            if 'yes' in response: 
                return "yes"
            elif 'no' in response: 
                return "no"
            else: 
                return "yes"
            
    def get_guess_variants(self, guess):
        trimmed_string = guess.lstrip()
        if not trimmed_string:
            return []
        if not trimmed_string[0].isalpha():
            return []
        uppercase_version = trimmed_string.capitalize()
        lowercase_version = trimmed_string[0].lower() + trimmed_string[1:]  
        return [uppercase_version, lowercase_version]

formatter = MistralPromptFormatter(inference_mode = True)


#################
# Define main function to call model and generate response

answer_bad_words = ['maybe', 'sometimes', 'Maybe', 'Sometimes']
guess_bad_words = ['answer', 'Answer']
guesses = []

def get_words_ids(words):
    return [tokenizer.encode(word, add_special_tokens=False) for word in words]

def tokenize(game_text, tokenizer):
    return tokenizer(game_text, return_tensors="pt")

def generate_similar_guess(guess):
    game_text = formatter.similar_guess_format(guess)
    game_text_len = len(game_text)
    game_tokens = tokenize(game_text, tokenizer)
    output_tokens = model.generate(input_ids=game_tokens["input_ids"].to("cuda"),
                                   attention_mask=game_tokens["attention_mask"].to("cuda"),
                                   pad_token_id = 2,
                                   do_sample = True,
                                   temperature = 0.9,
                                   top_k = 3,
                                   max_new_tokens=16)
    output = tokenizer.batch_decode(output_tokens.detach().cpu().numpy(), skip_special_tokens=False)[0]  
    response = output[game_text_len+3:]
    response = response.replace('</s>', '')
    response = response.replace('.','')
    return response

def generate_response(obs, guesses):
    game_text = formatter.format_obs(obs, guesses)
    game_text_len = len(game_text)
    game_tokens = tokenize(game_text, tokenizer)
    output_tokens = model.generate(input_ids=game_tokens["input_ids"].to("cuda"),
                                   attention_mask=game_tokens["attention_mask"].to("cuda"),
                                   pad_token_id = 2,
                                   max_new_tokens=64)
    output = tokenizer.batch_decode(output_tokens.detach().cpu().numpy(), skip_special_tokens=False)[0]  
    output = formatter.format_response(output, game_text_len, obs) 
    if obs.turnType == 'guess':
        if len(obs.questions) ==1:
            if obs.answers[-1] == 'yes':
                output = random.choice(['spring roll', 'smoked salmon', 'matcha', 'taco', 'sorbet', 'ravioli', 'knife sharpener', 'wok', 'mayonnaise']).lower()
                guesses.append(output)
                return output
            else:
                output = random.choice(['dog leash', 'fountain pen', 'chalk', 'mechanical keyboard', 'jet ski', 'toolbox', 'workbench', 'hammer', 'kimono', 'bicycle', 'loveseat', 'diploma']).lower()
                guesses.append(output)
                return output
        if len(obs.questions) ==2:
            if obs.answers[0] == 'yes' and obs.answers[1] == 'yes':
                output = random.choice(['spring roll', 'smoked salmon', 'taco', 'sorbet', 'ravioli', 'mayonnaise']).lower()
                guesses.append(output)
                return output
            if obs.answers[0] == 'yes' and obs.answers[1] == 'no':
                output = random.choice(['knife sharpener', 'wok', 'chopsticks', 'matcha', 'kombucha', 'dutch oven']).lower()
                guesses.append(output)
                return output
            if obs.answers[0] == 'no' and obs.answers[1] == 'yes':
                output = random.choice(['toolbox', 'workbench', 'hammer', 'drone', 'sewing machine']).lower()
                guesses.append(output)
                return output
            else:
                output = random.choice(['dog leash', 'fountain pen', 'chalk', 'mechanical keyboard', 'jet ski', 'kimono', 'bicycle', 'loveseat', 'diploma']).lower()
                guesses.append(output)
                return output
        output = output.lower()
        attempts = 0
        while output in (guesses+OLD_KEYWORDS) and attempts < 4:
            output = generate_similar_guess(output).lower()
            attempts += 1
        guesses.append(output)
    return output


#################
# Function to be called by the game environment

def agent_fn(obs, cfg):
    
    response = generate_response(obs, guesses)
    response = response.strip()
    
    #Catch glitched responses
    if (response is None) or (len(response) <= 1) or (len(response) >= 200):
        return "yes"    
    else:
        return response
    
def process_data(data):
    obs, cfg = pickle.loads(data)
    #print(f"received obs and cfg")
    result = agent_fn(obs, cfg)
    #print(f"result: {result}")
    return pickle.dumps(result)

print("Starting server...")

# Set up the server
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as server_socket:
    try:
        server_socket.bind((host, port))
        server_socket.listen()
        print(f"Server started, waiting for connection on port {port}...")
        while True:
            conn, addr = server_socket.accept()
            with conn:
                #print(f"Connected by {addr}")
                data = conn.recv(4096)
                if not data:
                    break
                result = process_data(data)
                conn.sendall(result)
    except OSError as e:
        print(f"Failed to start server: {e}")
        sys.exit(1)

Writing /tmp/submission/main2.py


In [5]:
from kaggle_environments import make
import kaggle_environments
import random

def random_agent(obs, cfg):
    if obs.turnType == "ask": response = "Is it a duck?"
    elif obs.turnType == "guess": response = "duck"
    elif obs.turnType == "answer": 
        random_int = random.randint(0, 1)
        if random_int: response = "Yes"
        else: response = "No"
    return response

class obs(object):
    def __init__(self, turnType, keyword, category, questions, answers):
        self.turnType = turnType
        self.keyword = keyword
        self.category = category
        self.questions = questions
        self.answers = answers
        
keyword = "argentina"
alts = ["argentina"]
kaggle_environments.envs.llm_20_questions.llm_20_questions.category = "Place"
kaggle_environments.envs.llm_20_questions.llm_20_questions.keyword_obj = {'keyword':keyword,'alts':alts}
kaggle_environments.envs.llm_20_questions.llm_20_questions.keyword = keyword
kaggle_environments.envs.llm_20_questions.llm_20_questions.alts = alts

env = make("llm_20_questions", debug=True)
#game_output = env.run(agents=["/tmp/submission/main.py", random_agent, random_agent, random_agent])
#env.render(mode="ipython", width=800, height=400)

No pygame installed, ignoring import


In [6]:
!apt install pigz pv > /dev/null

In [7]:
!tar --use-compress-program='pigz --fast --recursive | pv' -cf /kaggle/working/submission.tar.gz -C /tmp/submission .